In [1]:
import matplotlib
import sys
import os
current_dir = os.path.dirname(os.path.abspath('./'))
if not current_dir in sys.path:
    sys.path.append(current_dir)
    
from utils.structures import Pipeline, Deploy
from utils.data_management import dict2str
from typing import *
import tensorflow as tf
from sklearn.datasets import make_classification
import mne
from combiners import EpochsCombiner
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from utils.machine_learning.designer import ModelDesign, ParallelDesign, LayerDesign
from utils.machine_learning.analyzer import ModelAnalyzer, LFCNNAnalyzer
from mne.datasets import multimodal

# %matplotlib qt

2022-04-05 22:22:55.288443: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-05 22:22:55.288473: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
mne.set_log_level(verbose='CRITICAL')
fname_raw = os.path.join(multimodal.data_path(), 'multimodal_raw.fif')
raw = mne.io.read_raw_fif(fname_raw)

cond = raw.acqparser.get_condition(raw, None)
# get the list of condition names
condition_names = [k for c in cond for k,v in c['event_id'].items()]
epochs_list = [mne.Epochs(raw, **c) for c in cond]
epochs = mne.concatenate_epochs(epochs_list)
print(epochs.info)
epochs = epochs.pick_types(meg='grad')

In [ ]:
# print(dict2str(vars(epochs)))
print(epochs._channel_type_idx['grad'])
print(len(epochs._channel_type_idx['grad']))


In [4]:
X = np.array([])
Y = list()
for i, epochs in enumerate(epochs_list):
    data = epochs.get_data()
    if i == 0:
        X = data.copy()
    else:
        X = np.append(X, data, axis=0)
    Y += [i for _ in range(data.shape[0])]

Y = np.array(Y)


In [5]:
sample_matrix = X[0].copy()

X = np.array([X[i, epochs._channel_type_idx['grad'], :] for i, _ in enumerate(X)])


In [ ]:
plt.imshow(X[0, 0:10, 0:10], vmin=-1e-11, vmax=1e-11)
plt.show()

In [ ]:
for i, epochs in enumerate(epochs_list):
    epochs.plot()
    break

In [8]:
# one-hot encoding

def one_hot_encoder(Y: np.ndarray) -> np.ndarray:
    y = list()
    n_classes = len(np.unique(Y))

    for val in Y:
        new_y_value = np.zeros(n_classes)
        new_y_value[val] = 1
        y.append(new_y_value)

    return np.array(y)

y = list()
n_classes = len(np.unique(Y))

for val in Y:
    new_y_value = np.zeros(n_classes)
    new_y_value[val] = 1
    y.append(new_y_value)

y = np.array(y)


In [6]:
import random
from utils.console.colored import success

def shuffle_x_y(X: np.ndarray, Y: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    indices = [i for i in range(X.shape[0])]
    random.shuffle(indices)
    X = X[indices, :, :]
    Y = Y[indices]
    return X, Y


def test_shuffle_x_y():
    X = np.random.rand(10, 10, 10)
    Y = X[:, 0, 0]
    
    assert np.all(Y == X[:, 0, 0])
    
    i = np.random.randint(0, 9)
    X, Y = shuffle_x_y(X, Y)
    
    for i in range(10):
        assert np.all(Y[i] == X[i, 0, 0])
    
    success('All tests passed')
    

test_shuffle_x_y()


All tests passed


In [9]:
n_samples, n_channels, n_times = X.shape
n_classes = len(np.unique(Y))

lfcnn_design = ModelDesign(
    tf.keras.Input(shape=(n_channels, n_times, 1), name='input_layer'),
    tf.keras.layers.Conv2D(32, (n_channels, 1), activation=tf.nn.relu, name='spatial_filters_layer'),
    tf.keras.layers.DepthwiseConv2D((1, 15), padding='same', activation=tf.nn.relu, name='temporal_filters_layer'),
    LayerDesign(tf.transpose, perm=(0, 3, 2, 1)),
    tf.keras.layers.MaxPool2D(pool_size=(1, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(400, activation=tf.nn.relu),
    tf.keras.layers.Dense(200, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
)

In [10]:
import sklearn

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

In [ ]:
print(y_test.shape)
print(y_train.shape)

In [ ]:
from utils.machine_learning.analyzer import LFCNNAnalyzer

model = lfcnn_design.build()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['acc']
)

model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_split=0.4,
    shuffle=True,
)


model_analyzer = LFCNNAnalyzer(model)

model_analyzer.plot_metrics(['loss', 'val_loss'])
model_analyzer.plot_metrics(['acc', 'val_acc'])

In [ ]:
model_analyzer.plot_spatial_weights()
model_analyzer.plot_temporal_weights(transpose=True)
model_analyzer.plot_patterns(epochs.load_data().pick_types(meg='grad').info)

In [ ]:
import copy as cp
print(model_analyzer.layers['spatial_filters_layer'].data.shape)
p = model_analyzer.layers['spatial_filters_layer'].data
e = epochs.load_data().pick_types(meg='grad')
info = cp.deepcopy(e.info)

with info._unlock():
    info['sfreq'] = 1.

patterns = mne.EvokedArray(p, info, tmin=0)

patterns.plot_topomap(
    times=range(32),
    cmap='RdBu_r', sensors=True,
    colorbar=False, units='a.u.', res=64,
    size=1, cbar_fmt='%3.1f', time_format='Latent\nSource %01d',
    show=True, show_names=False, title=None,
    outlines='head', contours=6,
    image_interp='bilinear', average=None,
    sphere=None
)


In [23]:

y_pred = model(X_test).numpy()


In [ ]:

y_pred_int = np.round(y_pred, 1)

def one_hot_decoder(y: np.array) -> np.array:
    y_decoded = list()
    for val in y:
        y_decoded.append(np.where(val == val.max())[0][0])
    
    return np.array(y_decoded)
    
y_pred_decoded = one_hot_decoder(y_pred_int)
y_test_decoded = one_hot_decoder(y_test)
print('                 Actual prediction             |         Rounded prediction       |  Decoded | Actual')
for ap, rp, p, a in zip(y_pred, y_pred_int, y_pred_decoded, y_test_decoded):
    print('             ', np.round(ap, 3), '            ', rp, '        ', p, '       ', a)

In [16]:
n_samples, n_channels, n_times = X.shape
n_classes = len(np.unique(Y))

w = n_channels
h = n_times
c = 1

simple_conv_net = ModelDesign(
    tf.keras.Input(shape=(w, h, c), name='input_layer'),
    tf.keras.layers.Conv2D(32, (w//4, h//4), strides=(w//8, h//8), activation=tf.nn.relu),
    tf.keras.layers.Conv2D(16, (4, 4), activation=tf.nn.relu),
    tf.keras.layers.Conv2D(8, (2, 2), strides=(w//8, h//8), activation=tf.nn.relu),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(400, activation=tf.nn.relu),
    tf.keras.layers.Dense(200, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
)


In [ ]:
model = simple_conv_net.build()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['acc']
)

model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_split=0.4,
    shuffle=True,
)


model_analyzer = ModelAnalyzer(model)

model_analyzer.plot_metrics(['loss', 'val_loss'])
model_analyzer.plot_metrics(['acc', 'val_acc'])

In [ ]:
y_pred = model(X_test).numpy()
y_pred_int = np.round(y_pred, 1)

    

# for y_p, y in zip(y_pred_decoded, y_test_decoded):
#     print(y_p, y)

def one_hot_decoder(y: np.array) -> np.array:
    y_decoded = list()
    for val in y:
        y_decoded.append(np.where(val == val.max())[0][0])
    
    return np.array(y_decoded)
    
y_pred_decoded = one_hot_decoder(y_pred_int)
y_test_decoded = one_hot_decoder(y_test)
print('                 Actual prediction               |       Rounded prediction        | Decoded | Actual')
for ap, rp, p, a in zip(y_pred, y_pred_int, y_pred_decoded, y_test_decoded):
    print(np.round(ap, 3), rp, '    ', p, '     ', a)

In [25]:
slfcnn_design = ModelDesign(
    tf.keras.Input(shape=(n_channels, n_times, 1), name='input_layer'),
    tf.keras.layers.Conv2D(32, (n_channels, 1), activation=tf.nn.relu, name='spatial_filters_layer'),
    tf.keras.layers.DepthwiseConv2D((1, 15), padding='same', activation=tf.nn.relu, name='temporal_filters_layer'),
    LayerDesign(tf.transpose, perm=(0, 3, 2, 1)),
    tf.keras.layers.Conv2D(1, (32, 1), activation=tf.nn.relu, name='selection_filters_layer'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(400, activation=tf.nn.relu),
    tf.keras.layers.Dense(200, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
)

slfcnn_design().shape

TensorShape([None, 8])

In [ ]:
model = slfcnn_design.build()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['acc']
)

model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_split=0.4,
    shuffle=True,
)


model_analyzer = LFCNNAnalyzer(model)

model_analyzer.plot_metrics(['loss', 'val_loss'])
model_analyzer.plot_metrics(['acc', 'val_acc'])

In [ ]:
model_analyzer.plot_spatial_weights()
model_analyzer.plot_temporal_weights(transpose=True)
model_analyzer.plot_1d_weights(layer_identifier='selection_filters_layer', xlabel='latent sources')

In [ ]:
y_pred = model(X_test).numpy()
y_pred_int = np.round(y_pred, 1)

    

# for y_p, y in zip(y_pred_decoded, y_test_decoded):
#     print(y_p, y)

def one_hot_decoder(y: np.array) -> np.array:
    y_decoded = list()
    for val in y:
        y_decoded.append(np.where(val == val.max())[0][0])
    
    return np.array(y_decoded)
    
y_pred_decoded = one_hot_decoder(y_pred_int)
y_test_decoded = one_hot_decoder(y_test)
print('                 Actual prediction             |         Rounded prediction       |  Decoded | Actual')
for ap, rp, p, a in zip(y_pred, y_pred_int, y_pred_decoded, y_test_decoded):
    print('             ', np.round(ap, 3), '            ', rp, '        ', p, '       ', a)

In [ ]:
n_samples, n_channels, n_times = X.shape
n_classes = len(np.unique(Y))

cslfcnn_design = ModelDesign(
    tf.keras.Input(shape=(n_channels, n_times, 1), name='input_layer'), 
    tf.keras.layers.Conv2D(32, (n_channels, 1), activation=tf.nn.relu, name='spatial_filters_layer'),
    tf.keras.layers.DepthwiseConv2D((1, 15), padding='same', activation=tf.nn.relu, name='temporal_filters_layer'), 
    LayerDesign(tf.transpose, perm=(0, 3, 1, 2)),
    tf.keras.layers.DepthwiseConv2D((32, 1), padding='valid', activation=tf.nn.relu, name='selection_filters_layer'), 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(400, activation=tf.nn.relu),
    tf.keras.layers.Dense(200, activation=tf.nn.relu),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(50, activation=tf.nn.relu),
    tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
)

cslfcnn_design().shape

In [ ]:
model = cslfcnn_design.build()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['acc']
)

model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_split=0.4,
    shuffle=True,
)


model_analyzer = LFCNNAnalyzer(model)

model_analyzer.plot_metrics(['loss', 'val_loss'])
model_analyzer.plot_metrics(['acc', 'val_acc'])

In [ ]:
model_analyzer.plot_spatial_weights()
model_analyzer.plot_temporal_weights(transpose=True)

In [ ]:
# for layer_name, layer in model_analyzer.layers.items():
#     print(layer_name, layer.shape)

model_analyzer.plot_2d_weights('selection_filters_layer', title='Selection Weights', xlabel='Timepoints', ylabel='Latent Sources')

In [ ]:
selective_layer = model_analyzer.layers['selection_filters_layer']
# tmin=-0.1, tmax=0.5
print(epochs.times[180])
x = list()
y = list()
for i, weights in enumerate(selective_layer.data.T):
    x.append(np.round(epochs.times[i], 3))
    y.append(np.where(weights == weights.max())[0][0])
    # print(f'{np.round(epochs.times[i], 3)}: {np.where(weights == weights.max())[0][0]}')

from matplotlib.pyplot import figure

figure(figsize=(10, 8), dpi=80)
plt.plot(x, y, 'x')
plt.xlabel('Time (ms)')
plt.ylabel('Latent Sources')
plt.title('Maximum weighted latent sources over time')

In [9]:
n_samples, n_channels, n_times = X.shape
n_classes = len(np.unique(Y))

min_lfcnn_design = ModelDesign(
    tf.keras.Input(shape=(n_channels, n_times, 1), name='input_layer'),
    tf.keras.layers.Conv2D(32, (n_channels, 1), activation=tf.nn.relu, name='spatial_filters_layer'),
    tf.keras.layers.DepthwiseConv2D((1, 15), padding='same', activation=tf.nn.relu, name='temporal_filters_layer'),
    LayerDesign(tf.transpose, perm=(0, 3, 2, 1)),
    tf.keras.layers.MaxPool2D(pool_size=(1, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(.2),
    tf.keras.layers.Dense(n_classes, activation=tf.nn.softmax)
)

In [ ]:
model = min_lfcnn_design.build()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['acc']
)

model.fit(
    X_train,
    y_train,
    epochs=50,
    validation_split=0.4,
    shuffle=True,
)


model_analyzer = LFCNNAnalyzer(model)

model_analyzer.plot_metrics(['loss', 'val_loss'])
model_analyzer.plot_metrics(['acc', 'val_acc'])